In [ ]:
!pip install flyr
# !pip install exifread
# !pip install numpy
# !pip install flirimageextractor


In [17]:
import os
import csv
import json
import subprocess
import flyr  # pip install flyr

# --- CONFIGURATION ---

# 1. Path Setup
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    base_dir = os.getcwd()

INPUT_FOLDER = os.path.join(base_dir, "100_FLIR")
OUTPUT_CSV = os.path.join(base_dir, "Thermal_Data_Log.csv")

# 2. ExifTool Location
EXIFTOOL_PATH = os.path.join(base_dir, "exiftool-12.35.exe") 


# --- HELPER FUNCTIONS ---

def load_existing_records(csv_path):
    existing_signatures = set()
    
    if not os.path.exists(csv_path):
        return existing_signatures, False 

    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                ts = str(row.get("Timestamp", "")).strip()
                sn = str(row.get("Serial Number", "")).strip()
                if ts and sn:
                    existing_signatures.add((ts, sn))
        return existing_signatures, True
    except Exception as e:
        print(f"Warning: Could not read existing CSV ({e}). Starting fresh.")
        return existing_signatures, False

def get_folder_metadata_batch(folder_path, tool_path):
    print("Scanning metadata for all files...")
    tool_cmd = tool_path if os.path.exists(tool_path) else "exiftool"
    
    try:
        cmd = [
            tool_cmd, '-j', '-n', '-r',
            '-DateTimeOriginal', '-CameraModel', '-CameraSerialNumber',
            '-SerialNumber', '-Emissivity', '-ObjectDistance', 
            folder_path
        ]
        
        flags = 0
        if os.name == 'nt': 
            flags = subprocess.CREATE_NO_WINDOW

        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=flags)
        
        if not result.stdout:
            return []

        return json.loads(result.stdout)

    except Exception as e:
        print(f"Error in metadata extraction: {e}")
        return []

def process_incremental():
    headers = [
        "File Name", "Timestamp", "Camera Model", "Serial Number", 
        "Center Temp (C)", "Max Temp (C)", "Min Temp (C)", 
        "Avg Temp (C)", "Delta Temp (C)", 
        "Emissivity", "Distance", "Status"
    ]

    if not os.path.exists(INPUT_FOLDER):
        print(f"Error: Input folder not found: {INPUT_FOLDER}")
        return

    # 1. LOAD EXISTING DB
    existing_sigs, csv_exists = load_existing_records(OUTPUT_CSV)
    print(f"Found {len(existing_sigs)} existing records in CSV.")

    # 2. GET METADATA BATCH
    all_metadata = get_folder_metadata_batch(INPUT_FOLDER, EXIFTOOL_PATH)
    
    # 3. FILTER: Find only NEW images
    files_to_process = []
    
    for meta in all_metadata:
        # FORCE STRING CONVERSION (Fixes AttributeError)
        ts = str(meta.get("DateTimeOriginal", "")).strip()
        sn = str(meta.get("CameraSerialNumber", "")).strip()
        
        if not sn or sn == "None":
             sn = str(meta.get("SerialNumber", "")).strip()

        # Check if this exact shot is already in our CSV
        if (ts, sn) in existing_sigs:
            continue 
            
        files_to_process.append(meta)

    total_new = len(files_to_process)
    print(f"New images to process: {total_new}")
    
    if total_new == 0:
        print("Everything is up to date.")
        return

    # 4. PROCESS ONLY NEW FILES
    with open(OUTPUT_CSV, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        if not csv_exists:
            writer.writerow(headers)
        
        for i, meta in enumerate(files_to_process):
            full_path = meta.get('SourceFile')
            filename = os.path.basename(full_path)
            
            # --- PROGRESS INDICATOR ---
            # end='\r' returns cursor to start of line
            # " "*20 clears any leftover text from previous longer filenames
            print(f"Processing [{i+1}/{total_new}] {filename}" + " "*20, end='\r', flush=True)

            try:
                # --- A. Thermal Processing ---
                thermogram = flyr.unpack(full_path)
                thermal_data = thermogram.celsius
                
                min_t = thermal_data.min()
                max_t = thermal_data.max()
                avg_t = thermal_data.mean()
                delta_t = max_t - min_t
                
                h, w = thermal_data.shape
                center_t = thermal_data[h//2, w//2]

                # --- B. Prepare Data for Writing ---
                row_ts = str(meta.get("DateTimeOriginal", "Unknown"))
                row_model = str(meta.get("CameraModel", "Unknown"))
                
                row_sn = str(meta.get("CameraSerialNumber", ""))
                if not row_sn or row_sn == "None":
                    row_sn = str(meta.get("SerialNumber", "Unknown"))

                writer.writerow([
                    filename,
                    row_ts,
                    row_model,
                    row_sn,
                    f"{center_t:.1f}",
                    f"{max_t:.1f}",     
                    f"{min_t:.1f}",     
                    f"{avg_t:.1f}",
                    f"{delta_t:.1f}",
                    meta.get("Emissivity", "N/A"),
                    meta.get("ObjectDistance", "N/A"),
                    "Success"
                ])
                
                if i % 10 == 0: f.flush()
                
            except Exception as e:
                writer.writerow([filename, str(meta.get("DateTimeOriginal")), "", "", "", "", "", "", "", "", "", f"Error: {str(e)}"])

    # Clear the progress line and print done
    print("\n" + "-"*50)
    print(f"Done! Added {total_new} new records.")

if __name__ == "__main__":
    process_incremental()

Found 14 existing records in CSV.
Scanning metadata for all files...
New images to process: 2
Processing [2/2] FLIR0019.jpg                    
--------------------------------------------------
Done! Added 2 new records.


In [19]:
################################################### MAZEN  - DataFrame Viewer ##################################################
import pandas as pd

df = pd.read_csv("Thermal_Data_Log.csv")
df

,File Name,Timestamp,Camera Model,Serial Number,Center Temp (C),Max Temp (C),Min Temp (C),Avg Temp (C),Delta Temp (C),Emissivity,Distance,Status
0,FLIR0003.jpg,2025:11:24 14:30:18.107+02:00,FLIR E5 Pro,13333134,23.0,23.9,21.7,23.1,2.2,0.95,1,Success
1,FLIR0004.jpg,2025:11:24 14:31:19.177+02:00,FLIR E5 Pro,13333134,28.5,54.5,21.3,28.4,33.2,0.95,1,Success
2,FLIR0005.jpg,2025:11:24 14:40:36.898+02:00,FLIR E5 Pro,13333134,22.4,23.0,22.2,22.6,0.8,0.95,1,Success
3,FLIR0006.jpg,2025:11:24 15:19:36.703+02:00,FLIR E5 Pro,13333134,30.0,61.0,21.3,27.4,39.7,0.95,1,Success
4,FLIR0007.jpg,2025:11:25 09:59:39.012+02:00,FLIR E5 Pro,13333134,26.1,31.4,23.7,24.8,7.7,0.95,1,Success
5,FLIR0008.jpg,2025:11:25 10:25:57.424+02:00,FLIR E5 Pro,13333134,27.6,37.0,23.2,27.0,13.7,0.95,1,Success
6,FLIR0009.jpg,2025:11:25 10:26:10.676+02:00,FLIR E5 Pro,13333134,25.8,35.9,24.0,26.5,11.8,0.95,1,Success
7,FLIR0010.jpg,2025:11:25 14:19:38.198+02:00,FLIR E5 Pro,13333134,25.4,31.8,19.6,25.0,12.1,0.95,1,Success
8,FLIR0011.jpg,2025:11:25 14:21:46.913+02:00,FLIR E5 Pro,13333134,35.6,36.3,21.9,29.0,14.4,0.95,1,Success
9,FLIR0012.jpg,2025:11:26 09:47:52.605+02:00,FLIR E5 Pro,13333134,17.1,30.7,12.5,20.5,18.2,0.95,1,Success
